In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv", index_col="PassengerId")
test = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv", index_col="PassengerId")

In [ ]:
y = train["Survived"]
train.drop(columns=["Survived"], inplace=True)

In [ ]:
df = pd.concat([train,test])
df.isna().sum()

We are going to use only the variables Sex, Pclass, Embarked and Cabin.

In [ ]:
df.drop(columns=["Name","Age","SibSp","Parch","Ticket","Fare"], inplace=True)

## Fill Embarked and Cabin NaNs

In [ ]:
embarked_most_common = df.groupby(["Pclass","Sex"])["Embarked"].agg(lambda x:x.value_counts().index[0])
df.loc[df["Embarked"].isna(),"Embarked"] = pd.MultiIndex.from_frame(df[["Pclass","Sex"]])[df["Embarked"].isna()].map(embarked_most_common)

In [ ]:
df["Cabin"] = df["Cabin"].str[0]
df["Cabin"] = df["Cabin"].fillna("N")

In [ ]:
df[:len(train)].join(y).groupby("Cabin").agg({"Survived":["count","mean"]})

In [ ]:
df.loc[df["Cabin"]=="T","Cabin"] = "N"
df.loc[df["Cabin"]=="G","Cabin"] = "F"

# Predict with the Target Mean

We can calculate the target mean of the different subgroups of this categorical variables as you can see below.

In [ ]:
df[:len(train)].join(y).groupby(["Sex","Pclass","Embarked","Cabin"]).agg({"Survived":["count","mean"]})

The probability of survive of each subgroup is the target mean of that subgroup.

In [ ]:
mean = df[:len(train)].join(y).groupby(["Sex","Pclass","Embarked","Cabin"])["Survived"].mean()
df["target_mean"] = pd.MultiIndex.from_frame(df[["Sex","Pclass","Embarked","Cabin"]]).map(mean)

In [ ]:
df.isna().sum()

Some subgroups don't appear at train but only at test, thats why there are some NaNs in the predictions. To fill those NaNs we are going to do the same procedure without the Cabin variable.

In [ ]:
mean = df[:len(train)].join(y).groupby(["Sex","Pclass","Embarked"])["Survived"].mean()      
df.loc[df["target_mean"].isna(),"target_mean"] = pd.MultiIndex.from_frame(df[["Sex","Pclass","Embarked"]])[df["target_mean"].isna()].map(mean)

In [ ]:
df.isna().sum()

Now, calculate the accuracy and submit.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
df["prediction"] = df["target_mean"].round().astype(int)
accuracy_score(df.loc[:len(train)-1,"prediction"],y)

In [ ]:
sub = pd.read_csv("../input/tabular-playground-series-apr-2021/sample_submission.csv", index_col = "PassengerId")
sub["Survived"] = df.loc[len(train):,"prediction"]
sub.to_csv('sub_no_model.csv')